In [1]:
# imports
from astropy.io import fits
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import plotting as myP
import numpy as np
import csv 
# import match

In [2]:
# path variables 
path_cat = 'D:\IfA REU\Catalogs'
path_fig = 'D:\IfA REU\IfA-REU-Project\Figures'
path_csv = 'D:\IfA REU\IfA-REU-Project\Lists'

# Read CSV

In [3]:
# read from a given csv file and return a list of its contents 
def ReadFile(filename) : 
    # initialize list
    file = []
    # open csv file
    with open(path_csv+filename, newline='') as csv_file : 
        reader = csv.reader(csv_file, delimiter=',', quotechar='"')
        # output file into list for easy access
        for row in reader :
            file.append(row)
    # print column names 
    print(filename, '\tcolumns: ', file[0])
    # return an array of type int with column names clipped off
    return( np.array(file[1:], dtype=int) )

In [4]:
# read AGN ID and index from csv files
agn_irSel20 = ReadFile('\Match_COSMOS2020_IRAGN_to_COSMOS2020.csv')

# transpose arrays to get columns
agn_irSel20_id, agn_irSel20_i = agn_irSel20.T

\Match_COSMOS2020_IRAGN_to_COSMOS2020.csv 	columns:  ['ID', 'i']


# Read Catalog

In [5]:
# open COSMOS 2020 catalog file and get data 
inf20 = fits.open(path_cat+'\COSMOS2020_CLASSIC_R1_v2.0_master.fits')
data20 = inf20[1].data
cols20 = inf20[1].columns
head20 = inf20[0].header
inf20.close()

In [6]:
# print(cols20)

# Get Photometry 

In [7]:
# get photometry table values
flux_nu_uJy = np.array([   
        # The Galaxy Evolution Explorer (GALEX)
        np.array(data20['GALEX_FUV_FLUX']), 
        np.array(data20['GALEX_NUV_FLUX']),
        # Subaru / Hyper Suprime-Cam
        np.array(data20['HSC_g_FLUX_APER2']),
        np.array(data20['HSC_r_FLUX_APER2']),
        np.array(data20['HSC_i_FLUX_APER2']),
        np.array(data20['HSC_z_FLUX_APER2']),
        np.array(data20['HSC_y_FLUX_APER2']),
        # Paranal VISTA / VIRCAM
        np.array(data20['UVISTA_J_FLUX_APER2']),
        np.array(data20['UVISTA_H_FLUX_APER2']),
        np.array(data20['UVISTA_Ks_FLUX_APER2']),
        # Spitzer Large Area Survey with Hyper-Suprime-Cam (SPLASH) 
        np.array(data20['SPLASH_CH1_FLUX']),
        np.array(data20['SPLASH_CH2_FLUX']),
        np.array(data20['SPLASH_CH3_FLUX']),
        np.array(data20['SPLASH_CH4_FLUX']),
        # Herschel FIR
        np.array(data20['FIR_24_FLUX']),
        np.array(data20['FIR_100_FLUX']),
        np.array(data20['FIR_160_FLUX']),
        np.array(data20['FIR_250_FLUX']),
        np.array(data20['FIR_350_FLUX']),
        np.array(data20['FIR_500_FLUX'])
    ])  

# transpose so [i] is a source, not a column
flux_nu_uJy = flux_nu_uJy.T

# change -99 to NaN
flux_nu_uJy = np.where(flux_nu_uJy == float(-99), float('nan'), flux_nu_uJy)

# print info
print('Table shape:', np.shape(flux_nu_uJy))

Table shape: (1720700, 20)


In [8]:
# get table of agn sources 
agn_flux_uJy = flux_nu_uJy[agn_irSel20_i]

# print info
print('Table shape:', np.shape(agn_flux_uJy))

Table shape: (2287, 20)


In [9]:
# list wavelengths of the photometry filters (in Angstroms)
# -- Used lambda_ref from: http://svo2.cab.inta-csic.es/theory/fps3/
lam_A = np.array([
           1535.08,     # 'GALEX_FUV_FLUX'
           2300.79,     # 'GALEX_NUV_FLUX'
           4798.21,     # 'HSC_g_FLUX_APER2'
           6218.44,     # 'HSC_r_FLUX_APER2'
           7727.02,     # 'HSC_i_FLUX_APER2'
           8908.50,     # 'HSC_z_FLUX_APER2'
           9775.07,     # 'HSC_y_FLUX_APER2'
          12524.83,     # 'UVISTA_J_FLUX_APER2'
          16432.40,     # 'UVISTA_H_FLUX_APER2'
          21521.52,     # 'UVISTA_Ks_FLUX_APER2'
          35378.41,     # 'SPLASH_CH1_FLUX' 
          44780.49,     # 'SPLASH_CH2_FLUX' 
          56961.77,     # 'SPLASH_CH3_FLUX' 
          77978.39,     # 'SPLASH_CH4_FLUX' 
         240000.00,     # 'FIR_24_FLUX'         # vv -- are these accurate enough? I cant find them in SVO
        1000000.00,     # 'FIR_100_FLUX'
        1600000.00,     # 'FIR_160_FLUX'
        2500000.00,     # 'FIR_250_FLUX'
        3500000.00,     # 'FIR_350_FLUX'
        5000000.00      # 'FIR_500_FLUX'        # ^^ 
    ])

# print info 
print('Number of Filters: ', len(lam_A))

Number of Filters:  20


# Convert Units

In [10]:
# convert wavelength from angstrom to centimeter
lam_cm = lam_A * 10**(-8)

In [11]:
# speed of light (constant)
c_cm = 2.99792458 * 10**(10) # cm/s

# c/lambda^2
c_lam2 = c_cm / lam_cm**2

In [21]:
# convert Jy to cgs units: # 1 Jy = 10^-23 erg/s/cm2/Hz, uJy = 10^-6 Jy
agn_flux_nu_ergscm2Hz = agn_flux_uJy * 10**(-23) * 10**(-6)

# multiply F_nu by c/lam^2 to get F_lam
agn_flux_lam_ergscm2 = np.multiply(agn_flux_nu_ergscm2Hz, c_lam2)

# lam * F_lam [erg/s/cm]
agn_lamflux_lam_ergscm = np.multiply(agn_flux_lam_ergscm2, lam_cm)

# verify same shape 
print('Table shape:', np.shape(agn_lamflux_lam_ergscm))

Table shape: (2287, 20)


# Make SED